In [1]:
import os.path

import pandas as pd

from Config import config
from helper.helper import date_range_to_string

config.processing_date_range = date_range_to_string(start=pd.to_datetime('03-01-24'),
                                                    end=pd.to_datetime('09-01-24'))
# devided by rolling mean, std
n_mt_ohlcv = pd.read_csv(
    os.path.join(r"C:\Code\dl-forcasting\data\Kucoin\Spot\BTCUSDT",
                 f"n_mt_ohlcv.{config.processing_date_range}.csv.zip"), parse_dates=['date'], compression='zip')
n_mt_ohlcv.set_index(['timeframe', 'date'], inplace=True, drop=True)
n_mt_ohlcv.dtypes, n_mt_ohlcv.index.dtypes

DEBUG@10-17.19:35:23:...Starting


(open      float64
 close     float64
 high      float64
 low       float64
 volume    float64
 dtype: object,
 timeframe                 object
 date         datetime64[ns, UTC]
 dtype: object)

In [2]:
n_mt_ohlcv.describe()

,open,close,high,low,volume
count,340668.000000,340668.000000,340668.000000,340668.000000,340668.000000
mean,0.002571,0.002537,0.000586,0.004814,0.011528
std,1.080184,1.080047,1.081013,1.083676,1.070377
min,-11.829250,-11.796538,-10.789105,-12.271370,-4.911268
25%,-0.556785,-0.557024,-0.575532,-0.531490,-0.405402
50%,0.010991,0.010835,-0.014932,0.036296,-0.136793
75%,0.575594,0.574331,0.553501,0.591531,0.143095
max,13.730087,13.773854,13.686195,12.303279,15.969589


# Multi timeframe modelling


structure_timeframes = {
    '1W':{        pattern: '1D',        trigger: '4h',        double: '15min',    }, 
    '1D':{        pattern: '4h',        trigger: '1h',        double: '5min',    }, 
    '4h':{        pattern: '1h',        trigger: '15min',        double: '1min',    }, 
}
n_mt_ohlcv include open, high, low, close, and volume of all timeframes.
single_timeframe(n_mt_ohlcv, timeframe) will return data of specified timeframe.
using tensorflow create 4 parallel CNN-LSTM models each fed with structure, pattern, trigger, and double timeframe data.
join these parallel models together.


Check if model is not trained yet, try loading it from 'cnn_lstm_model.h5'. 
If it is already partially trained, or loaded from disk, continue training.
after completing training on each set of data save model into 'cnn_lstm_model.h5' to prevent loosing data in case of computer restart.


In [2]:
from training.trainer import mt_train_n_test

X, y = mt_train_n_test('4h', n_mt_ohlcv, model_input_lengths, batch_size=10)
X, y

DEBUG@10-17.19:39:22:"'t_open' in RollingMeanStdOHLCV but not in data:open           float64\nclose          float64\nhigh           float64\nlow            float64\nvolume         float64\npre_open       float64\nmean_open      float64\nstd_open       float64\nn_open         float64\npre_close      float64\nmean_close     float64\nstd_close      float64\nn_close        float64\npre_high       float64\nmean_high      float64\nstd_high       float64\nn_high         float64\npre_low        float64\nmean_low       float64\nstd_low        float64\nn_low          float64\npre_volume     float64\nmean_volume    float64\nstd_volume     float64\nn_volume       float64\ndtype: object"



KeyboardInterrupt



In [17]:
from training.trainer import plot_mt_train_n_test

plot_mt_train_n_test(X, y, n=1, base)


timeframe                 object
date         datetime64[ns, UTC]
dtype: object

In [31]:
from training.trainer import mt_train_n_test

a = mt_train_n_test('4h', n_mt_ohlcv, 10)
a

KeyError: 'value_column'

structure_timeframes = {
    '1W': {'pattern': '1D', 'trigger': '4h', 'double': '15min'},
    '1D': {'pattern': '4h', 'trigger': '1h', 'double': '5min'},
    '4h': {'pattern': '1h', 'trigger': '15min', 'double': '1min'}
}
loop over structure timeframe in ['1D', '4h']:
collect information from already prepared function read_ohlcv_features(start, end, timeframe)
create required iteration to pass data to train_model
choose double_timeframe_end in the range of start and end
structure_timeframe_end= structure_timeframe_end = trigger_timeframe_end = double_timeframe_end
calculate trigger_timeframe_start according to trigger_timeframe_end and number of bars shall be passed for taining 'trigger_model' 
calculate pattern_timeframe_start according to pattern_timeframe_end and number of bars shall be passed for taining 'pattern_model' 
calculate structure_timeframe_start according to structure_timeframe_end and number of bars shall be passed for taining 'structure_model' 

for:
```python
def create_cnn_lstm(input_shape, name_prefix):
    input_layer = Input(shape=input_shape)

    # CNN Layer with ReLU activation
    conv = Conv1D(filters=64, kernel_size=3, padding='same')(input_layer)
    conv = LeakyReLU()(conv)
    conv = Conv1D(filters=64, kernel_size=3, padding='same')(conv)
    conv = LeakyReLU()(conv)

    # Flatten the CNN output
    flatten = Flatten()(conv)

    # LSTM Layer (LSTM has built-in activations)
    lstm = LSTM(64, return_sequences=False)(tf.expand_dims(flatten, axis=1))

    # Fully connected layer with ReLU activation
    dense = Dense(64)(lstm)
    dense = LeakyReLU()(dense)

    return Model(inputs=input_layer, outputs=dense)

def build_model(input_shapes):
    structure_model = create_cnn_lstm((128, 5), 'structure_model')
    pattern_model = create_cnn_lstm((256, 5), 'pattern_model')
    trigger_model = create_cnn_lstm((256, 5), 'trigger_model')
    double_model = create_cnn_lstm((256, 5), 'double_model')
    
    combined_output = Concatenate()(
        [structure_model.output, pattern_model.output, trigger_model.output, double_model.output])
    
    # Add an additional Dense layer with ReLU activation
    combined_dense = Dense(128)(combined_output)
    combined_dense = LeakyReLU()(combined_dense)
    
    # Final output layer (for regression tasks, use linear activation; for classification, consider sigmoid/softmax)
    final_output = Dense(1, activation='linear')(combined_dense)
    
    # Define the final model
    model = Model(inputs=[structure_model.input, pattern_model.input, trigger_model.input, double_model.input],
                  outputs=final_output)
    
    # Compile the model with mean squared error loss for regression tasks
    model.compile(optimizer='adam', loss='mse')
    
    # Model summary
    model.summary()

    return model
def train_model(structure_data, pattern_data, trigger_data, double_data, target_data, input_shapes, model = None):
    '''
    Check if the model is already trained or partially trained. If not, build a new model. 
    Continue training_data the model and save the trained model to 'cnn_lstm_model.h5' after each session.

    Args:
        structure_data: Data for the structure timeframe.
        pattern_data: Data for the pattern timeframe.
        trigger_data: Data for the trigger timeframe.
        double_data: Data for the double timeframe.
        target_data: The labels or target values for training_data.
        input_shapes: A dictionary containing the input shapes for structure, pattern, trigger, and double timeframe data.
    Returns:
        The trained model.
    '''
    # Check if the model already exists, load if it does
    model_path = 'cnn_lstm_model.h5'
    
    if model is None:
        if os.path.exists(model_path):
            log_d("Loading existing model from disk...")
            model = load_model(model_path)
        else:
            log_d("Building new model...")
            model = build_model(input_shapes)

    # Train the model
    history = model.fit([structure_data, pattern_data, trigger_data, double_data],
                        target_data,
                        epochs=10,
                        batch_size=32)
    log_d(history)
    # Save the model after each training_data session to avoid losing progress
    model.save(model_path)
    log_d("Model saved to disk.")
    
    return model
```

In [ ]:
structure_timeframes = {
    '1W': {'pattern': '1D', 'trigger': '4h', 'double': '15min'},
    '1D': {'pattern': '4h', 'trigger': '1h', 'double': '5min'},
    '4h': {'pattern': '1h', 'trigger': '15min', 'double': '1min'}
}


def 